gpt2: decoder only model, so the encoder & the cross attention all missed

In [16]:
from dataclasses import dataclass
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

# ----------------------------------------------------------

@dataclass
class GPTConfig:
    block_size: int = 1024      # max sequence length
    vocab_size: int = 50257     # number of tokens: 50,000 BPE tokens + 256 bytes tokens + 1 <|end of the text|> token
    n_layer: int = 12           # number of layers
    n_head: int = 12            # number of heads
    n_embed: int = 768          # embedding dimensions
    
    
class CausalSelfAttention(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        assert config.n_embed % config.n_head == 0
        
        # key, query, value projection for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embed, 3 * config.n_embed)
        
        # output projection
        self.c_proj = nn.Linear(config.n_embed, config.n_embed)
        
        # regularization
        self.n_head = config.n_head
        self.n_embed = config.n_embed
        
        # not really a 'bias', more of a mask, but following the OPENAI/HF naming though
        self.register_buffer(
            'bias',
            torch.tril(
                torch.ones(config.block_size, config.block_size)
            ).view(1, 1, config.block_size, config.block_size)
        )

    def forward(self, x):
        B, T, C = x.size() # batch_size, sequence_length, embedding dimensionality (n_embed)
        # calculate query, key, value for all heads and move head forward to be the batch 
        # nh: number of heads; hs: head size, C: number of channels = nh * hs
        
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embed, dim=2)
        
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        
        # attention (materials the large (T, T) matrix for all queries and keys)
        attn = (q @ k.transpose(-2, -1)) * (1. / math.sqrt(k.size(-1)))
        attn = attn.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)
        
        y = attn @ v # (B, nh, T, T) * (B, nh, T, hs) -> (B. nh, T. hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        
        # output projection
        y = self.c_proj(y)
        
        return y 

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.c_fc = nn.Linear(config.n_embed, 4 * config.n_embed)
        self.gelu = nn.GELU(approximate='tanh') # a smoother relu
        self.c_proj = nn.Linear(4 * config.n_embed, config.n_embed)
        
    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embed)
        self.attn = CausalSelfAttention(config)
        # attention is  a communication operation, 
                        # a aggregation function, 
                        # a polling function, 
                        # a weighted sum function, 
                        # a reduce operation
        self.ln_2 = nn.LayerNorm(config.n_embed)
        self.mlp = MLP(config)
    
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x        

    
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embed),  # token embedding
            wpe = nn.Embedding(config.block_size, config.n_embed),  # positional encoding
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embed)
        ))
        
        self.lm_head = nn.Linear(config.n_embed, config.vocab_size, bias=False)
    
    def forward(self, idx):
        # idx is of shape(B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size"
        
        # forward the token and position embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embed)
        tok_emb = self.transformer.wte(idx) # token embedding of shape (B, T, n_embed)
        x = pos_emb + tok_emb
        
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
            
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        return logits 
        
    
    
    @classmethod
    def from_pretrain(cls, model_type):
        '''loads pretrained GPT-2 model weights from huggingface'''
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print(f"loading weights from pretrained gpt: {model_type}")
        
        # n_layer, n_head, n_embed are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embed=768), # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embed=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embed=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=48, n_embed=1600), # 1558M params
        }[model_type]
        
        config_args['vocab_size'] = 50257   # always
        config_args['block_size'] = 1024    # always
        config = GPTConfig(**config_args)
        model = GPT(config)
        
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask
        
        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()
        
        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])
        
        return model

In [17]:
num_return_sequences = 5
max_length = 30
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GPT.from_pretrain('gpt2')
# 随机初始化的GPT
model = GPT(GPTConfig())
model.eval()
model.to(DEVICE)
print("didn't crash yay!")

# prefix tokens
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(DEVICE)

# generate right now x is (B, T) where B = 5, T = 8
torch.manual_seed(42)
torch.cuda.manual_seed(42)

while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x) # (B, T, vocab_size)
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top k sampling of 50 (higgingface pipeline default)
        # top-k _probs here becomes (5, 50), topk-indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)
        
# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)
    



didn't crash yay!
> Hello, I'm a language model Di Pavilionirst Aminashedimum Warcraft whilstWhy.)UN ratings tert Quin bos harmony352 analyses clos Kill Dispiao Against
> Hello, I'm a language model containment VIEW wrapping Mexicans Pair Globeague attaches brochSTR insights pull Canyonignment legendary Parliamentary ts smuggledivating screenshot Phil For association
> Hello, I'm a language model fingertips Mn baitFind Ravens Original cavalry blazeotom rosesBreakoman flared slander To Failency Giuliani McGr hopesettings Akin Heather
> Hello, I'm a language model restaurantsット frameworks agencies convict dreadedaith tempting Beirutremoveiled Erit Terra IcGer Navyhusband Neigh 451 Lazarus bip indicating Henderson
> Hello, I'm a language modeluta Yo cac Burg PulitzerspectionCredit watershed uncover unlimited667 migrating babeMahN gly clustered autobiography Vikingorting104 winatson
